In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import pymysql
import re
from dotenv import load_dotenv
load_dotenv()
import os

In [2]:
db_connection_str = os.getenv("DB_CONNECTION_STR")

db_connection = create_engine(db_connection_str)

df = pd.read_sql('SELECT * FROM crash_db_Nashville', con=db_connection)
df = df.replace('',np.nan)

In [3]:
df.shape

(34692, 29)

In [4]:
df.head()

,cid,createDate,createBy,dataFrom,Accident_Number,Date_and_Time,Number_of_Motor_Vehicles,Number_of_Injuries,Number_of_Fatalities,Property_Damage,...,Harmful_Code_Description,Street_Address,City,State,ZIP,RPA,Precinct,Latitude,Longitude,Mapped_Location
0,1000000000,2020-03-06 08:21:31,David,Nashville,20190038972,01/15/2019 07:40:00 PM,2,0,0,None,...,None,None,ANTIOCH,TN,37013,8753,SOUTH,36.0449,-86.6671,None
1,1000000001,2020-03-06 08:21:31,David,Nashville,20190045402,01/17/2019 11:09:00 PM,2,0,0,None,...,None,None,NASHVILLE,TN,37214,8955,HERMIT,36.1531,-86.6291,None
2,1000000002,2020-03-06 08:21:31,David,Nashville,20190051468,01/20/2019 12:57:00 PM,2,0,0,None,...,None,None,NASHVILLE,TN,37204,8305,MIDTOW,36.1122,-86.7625,None
3,1000000003,2020-03-06 08:21:31,David,Nashville,20190088097,02/02/2019 12:38:00 AM,2,0,0,None,...,None,None,NASHVILLE,TN,None,None,None,36.0483,-86.4369,None
4,1000000004,2020-03-06 08:21:31,David,Nashville,20190091289,02/03/2019 01:25:00 PM,2,0,0,None,...,None,None,NASHVILLE,TN,37220,8615,MIDTOW,36.0618,-86.7405,None


In [5]:
df.columns

Index(['cid', 'createDate', 'createBy', 'dataFrom', 'Accident_Number',
       'Date_and_Time', 'Number_of_Motor_Vehicles', 'Number_of_Injuries',
       'Number_of_Fatalities', 'Property_Damage', 'Hit_and_Run',
       'Reporting_Officer', 'Collision_Type_Code',
       'Collision_Type_Description', 'Weather_Code', 'Weather_Description',
       'Illumination_Code', 'Illumination_Description', 'Harmful_Code',
       'Harmful_Code_Description', 'Street_Address', 'City', 'State', 'ZIP',
       'RPA', 'Precinct', 'Latitude', 'Longitude', 'Mapped_Location'],
      dtype='object')

In [6]:
df2 = pd.DataFrame()
df2['cid'] = df.cid
df2['city'] = df.dataFrom
df2['latitude'] = df.Latitude
df2['longitude'] = df.Longitude
df2['crash_time'] = pd.to_datetime(df.Date_and_Time)
df2['street_name'] = df.Street_Address
df2['injured_count'] = df.Number_of_Injuries.astype('int') + df.Number_of_Fatalities.astype('int')
df2['crash_date'] = pd.to_datetime(df2['crash_time'].dt.date)

In [7]:
df2.head()

,cid,city,latitude,longitude,crash_time,street_name,injured_count,crash_date
0,1000000000,Nashville,36.0449,-86.6671,2019-01-15 19:40:00,None,0,2019-01-15
1,1000000001,Nashville,36.1531,-86.6291,2019-01-17 23:09:00,None,0,2019-01-17
2,1000000002,Nashville,36.1122,-86.7625,2019-01-20 12:57:00,None,0,2019-01-20
3,1000000003,Nashville,36.0483,-86.4369,2019-02-02 00:38:00,None,0,2019-02-02
4,1000000004,Nashville,36.0618,-86.7405,2019-02-03 13:25:00,None,0,2019-02-03


In [8]:
df2.to_sql('cleaned_crash_Nashville',con = db_connection, if_exists="replace",index=False)

In [9]:
df2.to_sql('home_allcleanedcrashdata',con = db_connection, if_exists="append",index=False)